<a href="https://colab.research.google.com/github/datavzch/GANWS21_DEMO/blob/main/Generating_from_Stylegan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Requirements and Preparation**

In [ ]:
%tensorflow_version 1.x

In [ ]:
%pip install Google-Colab-Transfer

**Mount Google Drive**

In [ ]:
import colab_transfer

gd = colab_transfer.get_path_to_home_of_google_drive()
lm = colab_transfer.get_path_to_home_of_local_machine()

colab_transfer.mount_google_drive()

**Clone Stylegan2-ada  Repository**

In [ ]:
%cd /content/
%rm -rf stylegan2-ada/
!git clone https://github.com/NVlabs/stylegan2-ada.git
%cd stylegan2-ada/
!git checkout google-colab
%cd /content/

**Copy Dataset**

In [ ]:
folder_name = 'datasets/dataset/'
colab_transfer.copy_folder(folder_name)

**Create link with google drive**

In [ ]:
!ln -s "/content/drive/My Drive/" "/content/gdrive"

In [ ]:
gdrive_root = '/content/gdrive/'
result_root_folder = 'training-runs/'

In [ ]:
result_folder_name_to_load = '00025-steam-mirror-auto_no_ramp1-kimg5000-p0.713-resumecustom-freezed10/'
network_base_name='network-snapshot-005000.pkl'

In [ ]:
from pathlib import Path

network_pkl = f'{gdrive_root}{result_root_folder}{result_folder_name_to_load}{network_base_name}'

print('{}\nFile exists? {}'.format(
    network_pkl,
    Path(network_pkl).exists(),
))

In [ ]:
%cd /content/

# **Generate average Image**

In [ ]:
%cd ./stylegan2-ada/

In [ ]:
import dnnlib
import pickle
from dnnlib import tflib

dnnlib.tflib.init_tf()
print('Loading networks from "%s"...' % network_pkl)
with dnnlib.util.open_url(network_pkl) as fp:
    _, _, Gs = pickle.load(fp)
dlatents_var = Gs.get_var('dlatent_avg')

In [ ]:
import numpy as np
dlatents = np.tile(dlatents_var, [1, Gs.components.synthesis.input_shape[1], 1])

In [ ]:
import dnnlib
import dnnlib.tflib as tflib

Gs_syn_kwargs = dnnlib.EasyDict()
Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8,
                                      nchw_to_nhwc=True)
Gs_syn_kwargs.randomize_noise = False

In [ ]:
assert dlatents.shape[1:] == (14, 512) # [N, 14, 512]
image = Gs.components.synthesis.run(dlatents, **Gs_syn_kwargs)[0]

In [ ]:
from pathlib import Path
import PIL.Image

fname = Path(network_pkl).name.replace('.pkl', '.jpg')
img = PIL.Image.fromarray(image, 'RGB')
img.save(fname)
img

# **Generate random Image**

In [ ]:
%cd ../

In [ ]:
num_images = 5

In [ ]:
!python stylegan2-ada/generate.py --network={network_pkl} --outdir=out --trunc=1.0 --seeds=0-{num_images-1}

In [ ]:
!python stylegan2-ada/generate.py --network={network_pkl} --outdir=out --trunc=0.5 --seeds=0-{num_images-1}

# **Generate style mixing**

In [ ]:
!python stylegan2-ada/style_mixing.py --network={network_pkl} --outdir=out --trunc=1.0 --rows=1,26,128,318,428,938 --cols=1,28,128,318,428,938

In [ ]:
!python stylegan2-ada/style_mixing.py --network={network_pkl} --outdir=out --trunc=0.5 --rows=1,28,128,318,428,938 --cols=1,28,128,318,428,938

In [ ]:
#Add Latent Walk Generation